In [ ]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.by import By

from os import path, rename
import shutil
import re
import random
import webbrowser


In [ ]:
# Scrape Name of hospital and Link
# //h2/a
# Scrape Location
# //section/div/div/div[*]/p

In [ ]:
# treat empty strings and np.nan as NAm will impact dropna method
pd.options.mode.use_inf_as_na = True
# set Jupyter to display all output from a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
MAX_USER_PROFILES = 30

In [ ]:
df_allDocDetails = pd.read_csv("df_allDocDetails_complete.csv")

df_allDocDetails.shape
df_allDocDetails.drop_duplicates(inplace=True)
df_allDocDetails.shape
COMPLETED_URLS = list(set(list(df_allDocDetails.URL.values)))
len(COMPLETED_URLS)
with open('EXCEPTION_URLS.txt', 'r') as filehandle:
    EXCEPTION_URLS = [current_place.rstrip() for current_place in filehandle.readlines()]
len(EXCEPTION_URLS)
with open('NO_DATA_URLS.txt', 'r') as filehandle:
    NO_DATA_URLS = [current_place.rstrip() for current_place in filehandle.readlines()]
len(NO_DATA_URLS)


In [ ]:
EXCEPTION_URLS = list(set(EXCEPTION_URLS).difference(set(COMPLETED_URLS)))
EXCEPTION_URLS = list(set(EXCEPTION_URLS).difference(set(NO_DATA_URLS)))

len(EXCEPTION_URLS)


In [ ]:
#data_dir = './sampleAppExtension/usNews/rankingData/'
#download_dir = '/Users/rajivdulepet/Downloads/usNews/adultDoctorData/'
#results_dir = './sampleAppExtension/usNews/adultDoctorData/'
#results_dir = './sampleAppExtension/usNews/childrenDoctorData/'
#results
#download_file = download_dir + 'data.txt'
results_dir = './results/'


In [ ]:
Cancer_AdultHospitalRanking = './Cancer_AdultHospitalRanking.json'
All_ChildrenHospital =  './All_ChildrenHospital.json'
Cancer_ChildrenHospitalRanking = './Cancer_ChildrenHospitalRanking.csv'

In [ ]:
df_Cancer_AdultHospitalRanking = pd.read_json(Cancer_AdultHospitalRanking)
df_All_ChildrenHospital = pd.read_json(All_ChildrenHospital)
df_Cancer_ChildrenHospitalRanking = pd.read_csv(Cancer_ChildrenHospitalRanking)


In [ ]:
df_Cancer_AdultHospitalRanking['Ranking'] = df_Cancer_AdultHospitalRanking.index + 1
df_Cancer_ChildrenHospitalRanking['Ranking'] = df_Cancer_ChildrenHospitalRanking.index + 1

In [ ]:
df_Cancer_ChildrenHospitalRanking['Link'] =  'https://health.usnews.com' + df_Cancer_ChildrenHospitalRanking['Link']

In [ ]:
len(set(list(df_Cancer_ChildrenHospitalRanking.Link.values)).difference(set(list(df_All_ChildrenHospital.Link.values))))



In [ ]:
df_allDocDetails['Hospital Link'] = df_allDocDetails['URL'].str.replace('/doctors', '')


In [ ]:
df_allDocDetails.columns
df_Cancer_ChildrenHospitalRanking.columns
df_Cancer_AdultHospitalRanking.columns

In [ ]:
df_Cancer_ChildrenHospitalRanking.columns = ['Hospital', 'Hospital Link', 'Location', 'Ranking']
df_Cancer_AdultHospitalRanking.columns = ['Hospital', 'Hospital Link', 'Location', 'Ranking']
df_allDocDetails.rename(columns={'URL':'Doctors Link'}, inplace=True)

In [ ]:
df_allDocDetails.columns
df_Cancer_ChildrenHospitalRanking.columns
df_Cancer_AdultHospitalRanking.columns

In [ ]:
df_Cancer_ChildrenHospitalRanking['Type'] = 'Chidren'
df_Cancer_AdultHospitalRanking['Type'] = 'Adult'


In [ ]:
df_allDocDetails = pd.merge(df_allDocDetails, 
                            df_Cancer_AdultHospitalRanking, 
                            left_on=['Hospital Link'], 
                            right_on=['Hospital Link'], 
                            how='left')


In [ ]:
df_allDocDetails.columns

In [ ]:
df_allDocDetails.drop(columns=['Hospital_y', 'Location_y'], inplace=True)
df_allDocDetails.rename(columns={'Hospital_x':'Hospital', 'Location_x':'Location'}, inplace=True)

df_allDocDetails.columns

In [ ]:
df_allDocDetails = pd.merge(df_allDocDetails, df_Cancer_ChildrenHospitalRanking, left_on=['Hospital Link'], right_on=['Hospital Link'], how='left')


In [ ]:
df_allDocDetails.columns

In [ ]:
df_allDocDetails.drop(columns=['Hospital_y', 'Location_y'], inplace=True)
df_allDocDetails.rename(columns={'Hospital_x':'Hospital', 'Location_x':'Location'}, inplace=True)

df_allDocDetails.columns

In [ ]:
df_allDocDetails['Ranking_x'] = df_allDocDetails.apply(lambda row: row['Ranking_y'] if pd.notna(row['Ranking_y']) else row['Ranking_x'], axis=1)


In [ ]:
df_allDocDetails.drop(columns=['Ranking_y'], inplace=True)
df_allDocDetails.rename(columns={'Ranking_x':'Ranking'}, inplace=True)

df_allDocDetails.columns

In [ ]:
df_allDocDetails['Type_x'] = df_allDocDetails.apply(lambda row: row['Type_y'] if pd.notna(row['Type_y']) else row['Type_x'], axis=1)
df_allDocDetails.drop(columns=['Type_y'], inplace=True)
df_allDocDetails.rename(columns={'Type_x':'Type'}, inplace=True)
df_allDocDetails.columns


In [ ]:
df_allDocDetails[df_allDocDetails.Ranking==913].Hospital.unique()

In [ ]:
df_allDocDetails.to_csv('df_allDocDetails_with_ranking.csv', index=False)


In [ ]:
df_allDocDetails = df_allDocDetails[pd.notna(df_allDocDetails['Ranking'])]
df_allDocDetails['Ranking'] = df_allDocDetails['Ranking'].apply(lambda x: int(x) if pd.notna(x) else None)

In [ ]:
df_allDocDetails[df_allDocDetails.Ranking==914]['Hospital'].unique()

In [ ]:
df_allDocDetails[pd.notna(df_allDocDetails.Ranking)].to_csv('df_allDocDetails_with_cancer_ranking.csv', index=False)


In [ ]:
df_allDocDetails = pd.read_csv('df_allDocDetails_with_cancer_ranking.csv')
df_allDocDetails.shape
df_allDocDetails.columns
df_allDocDetails[df_allDocDetails.Ranking==1].Hospital.unique()
df_allDocDetails[df_allDocDetails.Ranking==50].Hospital.unique()
df_allDocDetails[df_allDocDetails.Ranking==95].Hospital.unique()
df_allDocDetails[df_allDocDetails.Ranking==913].Hospital.unique()

In [ ]:
chrome_options = Options()
chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
#Change chrome driver path accordingly
chrome_driver = "chromedriver"
driver = webdriver.Chrome(chrome_driver, chrome_options=chrome_options)

#time.sleep(2)
driver.execute_script("window.open('');")
time.sleep(2)


In [ ]:
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time

def delete_cache(pageDriver):
    pageDriver.switch_to.window(pageDriver.window_handles[1])
    time.sleep(2)
    pageDriver.get('chrome://settings/clearBrowserData') # for old chromedriver versions use cleardriverData
    time.sleep(2)
    actions = ActionChains(pageDriver) 
    actions.send_keys(Keys.TAB * 9 + Keys.DOWN * 9 + Keys.ENTER) # send right combination
    actions.perform()
    #time.sleep(2)
    #actions = ActionChains(pageDriver) 
    #actions.send_keys(Keys.TAB * 4 + Keys.ENTER) # confirm
    #actions.perform()
    #time.sleep(5) # wait some time to finish
    #pageDriver.close() # close this tab
    time.sleep(5) # wait some time to finish
    pageDriver.switch_to.window(pageDriver.window_handles[0]) # switch back
    time.sleep(2)

#delete_cache()


In [ ]:
def loadMainHospitalDoctorPage(pageDriver, url):
    pageDriver.switch_to.window(pageDriver.window_handles[1])
    time.sleep(3)
    pageDriver.get(url)
    time.sleep(3)


In [ ]:
def loadAllDoctorsPages(pageDriver):
    #pageDriver.switch_to.window(driver.window_handles[1])
    #time.sleep(3)
    # keep loading more until everything is loaded on page
    while (True):
        #buttons = pageDriver.find_elements(By.XPATH, '//*[contains(text(), "Load More")]')
        buttons = WebDriverWait(pageDriver, 20).until(EC.visibility_of_all_elements_located((By.XPATH, '//*[contains(text(), "Load More")]')))
        if len(buttons) > 1:
            pageDriver.execute_script("arguments[0].scrollIntoView();", buttons[0])
            time.sleep(3)
            pageDriver.execute_script("arguments[0].click();", buttons[0])
            time.sleep(3)
            
            foundLoadMoreError = False
            try:
                loadMoreError = pageDriver.find_element(By.CSS_SELECTOR, '[class*=message-box__Icon]').find_elements(By.XPATH, './/span')
                foundLoadMoreError = True
            except:
                foundLoadMoreError = False
                pass
            finally:
                if foundLoadMoreError:
                    raise
        else:
            break

In [ ]:
def extractDoctorsData(pageDriver):
    #pageDriver.switch_to.window(driver.window_handles[1])
    #time.sleep(3)
    hospitalName = pageDriver.find_element(By.XPATH, '//h1').text
    hospitalName = re.sub(r'\s+Doctors$', '', hospitalName)
    
    locationElement = pageDriver.find_element(By.XPATH, './/div[contains(@class, "Villain__Content")]').find_element(By.XPATH, './/p[contains(@class, "Paragraph-sc")]')
    location = locationElement.text
    location = re.sub(r'\s*\|.*', '', location)
    
    allDoctors = pageDriver.find_elements(By.CSS_SELECTOR, '[data-test-id=DetailCardDoctor]')
    
    docAllRecords = []
    
    for doc in allDoctors:
        docName = doc.find_element(By.XPATH, './/h2').text
        
        
        specialitiesElements = doc.find_element(By.CSS_SELECTOR, '[class*=DetailCardDoctor__Wrapper]').find_elements(By.XPATH, './/p');
        broader_speciality = ''
        specific_specialities = []
        
        # specialities
        if len(specialitiesElements) > 1:
            broader_speciality = specialitiesElements[0].text
            
            specialityContent = specialitiesElements[1].text
            individualSpecialities = specialityContent.split(',')
            individualSpecialities = [x.strip() for x in individualSpecialities]
            specific_specialities.extend(individualSpecialities)
        elif len(specialitiesElements) == 1:
            broader_speciality = specialitiesElements[0].text
        
        bio = doc.find_element(By.CSS_SELECTOR, 'p.sm-hide').text
        
        docRecord = {
            'URL': pageDriver.current_url, 
            'Hospital': hospitalName, 
            'Location': location, 
            'Doctor': docName, 
            'Broader Specialty': broader_speciality, 
            'Specific Specialty': specific_specialities, 
            'Bio': bio
            }
        
        try:
            additionalAttributes = doc.find_elements(By.XPATH, './/div[contains(@class, "DetailCardDoctor__DataPoint")]')
            
            if (additionalAttributes):
                for attribute in additionalAttributes:
                    paragraphs = attribute.find_elements(By.XPATH, './/p')
                    if paragraphs:
                        try:
                            if (paragraphs[1].text != '') and (paragraphs[0].text != ''):
                                docRecord[paragraphs[1].text] = paragraphs[0].text
                        except:
                            pass
        except Exception as inst:
            #print(type(inst))    # the exception instance
            #print(inst.args)     # arguments stored in .args
            pass

        docAllRecords.append(docRecord)


    return docAllRecords


%%time
rounds = 1
while(True):
    print(f'ROUNDS={rounds}')
    target_url = 'https://health.usnews.com/best-hospitals/area/oh/cleveland-clinic-6410670/doctors'
    loadMainHospitalDoctorPage(driver, target_url)
    #docDetails = extractDoctorsData(driver)
    #docDetails
    loadAllDoctorsPages(driver)
    rounds += 1

In [ ]:
%%time
#EXCEPTION_URLS = []
#NO_DATA_URLS = []
allDocDetails = []

for index, row in df_all_hospitals_json.iterrows():
    # show the doctor page and that should automatically trigger chrome extension to parse page and extract results
    target_url = row['Link']+'/doctors'
    
    #if (target_url in COMPLETED_URLS) or (target_url in EXCEPTION_URLS) or (target_url in NO_DATA_URLS):
    if (target_url not in EXCEPTION_URLS):
        continue
    print(index, row['Hospital'])
    try:
        loadMainHospitalDoctorPage(driver, target_url)
        loadAllDoctorsPages(driver)
        docDetails = extractDoctorsData(driver)
        
        print(f'\tNUM_RECORDS={len(docDetails)}')
        
        if len(docDetails) == 0:
            NO_DATA_URLS.append(target_url)
            print(f'\tNODATA={target_url}')
            delete_cache(driver)
        else:
            allDocDetails.extend(docDetails)
            delete_cache(driver)
    except:
        delete_cache(driver)
        EXCEPTION_URLS.append(target_url)
        print(f'\tEXCEPTION={target_url}')

In [ ]:
# <span class="message-box__Icon-sc-7sukoj-1 iVWJxw"></span>
# https://health.usnews.com/best-hospitals/area/mo/st-louis-university-hospital-6631020/doctors
len(set(EXCEPTION_URLS))
len(set(NO_DATA_URLS))
len(allDocDetails)
#allDocDetails

In [ ]:
df_allDocDetails = pd.DataFrame(allDocDetails)
df_allDocDetails.columns
df_allDocDetails.shape


In [ ]:
#df_allDocDetails.to_csv("df_allDocDetails_Feb14_943AM.csv", index=False)
df_allDocDetails.to_csv("df_allDocDetails_Feb19_1017PM.csv", index=False)
with open('NO_DATA_URLS.txt', 'w') as filehandle:
    filehandle.writelines("%s\n" % place for place in NO_DATA_URLS)
with open('EXCEPTION_URLS.txt', 'w') as filehandle:
    filehandle.writelines("%s\n" % place for place in EXCEPTION_URLS)

In [ ]:
# INITIALIZE
chrome_options = Options()
chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
#Change chrome driver path accordingly
chrome_driver = "chromedriver"
driver = webdriver.Chrome(chrome_driver, chrome_options=chrome_options)

#time.sleep(2)
driver.execute_script("window.open('');")
time.sleep(2)
#driver.switch_to.window(driver.window_handles[0]) # switch back
#time.sleep(2)

#delete_cache(driver)
#delete_cache(driver)



In [ ]:
## FIRST RUN CHROME DEBUG SESSION on TERMINAL using default profile, CLOSE EXISTING BROWSER BEFORE DOING IT
# Google\ Chrome --remote-debugging-port=9222 --user-data-dir="/Users/rajivdulepet/Library/Application Support/Google/Chrome" "https://google.com"


In [ ]:
%%time
def wait_for_extension_data_output(temp_file):
    # max wait time in seconds
    MAX_WAIT_TIME = 240

    timeout_counter = MAX_WAIT_TIME
    while not path.exists(temp_file):
        time.sleep(1)
        timeout_counter -= 1
        if timeout_counter == 0:
            break

    if path.isfile(temp_file):
        # just in case wait for the file to be complete and written by extension
        # using heuristic of 15 seconds
        time.sleep(15)
        return True
    else:
        return False

# wait_for_extension_data_output(download_file)

In [ ]:
IGNORE_HOSPITALS = ['Hospital HIMA San Pablo Oncológico', 
                    'Lakeland Regional Health Medical Center', 
                    'Auxilio Mutuo Hospital', 
                    'UofL Health-Jewish Hospital', 
                    'North Memorial Health Hospital', 
                    'University Hospital-San Juan', 
                    'Hospital HIMA San Pablo Bayamon', 
                    'Doctors Center', 
                    'Albert J. Solnit Psychiatric Center-South Campus', 
                   ]


In [ ]:
driver.switch_to.window(driver.window_handles[-1])
time.sleep(3)
driver.get('https://health.usnews.com/best-hospitals/area/az/mayo-clinic-6860019/doctors')


In [ ]:
MyFile=open('IGNORE_HOSPITALS.csv','w')
MyFile.writelines(IGNORE_HOSPITALS)
MyFile.close()

In [ ]:
%%time
done = False
hit_exception = False

while not done:
    #chrome_options = Options()
    #chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
    #Change chrome driver path accordingly
    #chrome_driver = "chromedriver"
    #driver = webdriver.Chrome(chrome_driver, chrome_options=chrome_options)

    # https://health.usnews.com/best-hospitals/area/az/mayo-clinic-6860019/doctors
    for index, row in df_all_hospitals_json.iterrows():
        destination_file = row['Hospital'].replace("'", '').replace('"', '').replace('\\', '-').replace('/', '-')
        destination_file = 'doctors-' + re.sub(r'\s+', '-', destination_file)
        destination_file = results_dir + destination_file + '.json'

        # check if we processed earlier so we can continue to the next one
        if path.exists(destination_file) and path.isfile(destination_file):
            # then skip and move to next hospital
            continue

        # launch browser instance with different profile each time to avoid getting blocked
        #chrome_options.add_argument('--profile-directory=Default')
        # /Users/rajivdulepet/Library/Application Support/Google/Chrome/Default
        #profileIndex = random.choice(range(1,MAX_USER_PROFILES+1))
        #profileIndex = 1
        #print(f'current browser profile = /Users/rajivdulepet/Desktop/aws/~/ChromeProfile{profileIndex}')
        #chrome_options.add_argument(f"user-data-dir=/Users/rajivdulepet/Desktop/aws/~/ChromeProfile{profileIndex}") #Path to your chrome profile
        #chrome_options.add_argument(f"user-data-dir=/Users/rajivdulepet/Library/Application Support/Google/Chrome") #Path to your chrome profile

        #chrome_options.add_extension('/Users/rajivdulepet/Desktop/aws/sampleAppExtension/myext.zip')
        #driver = webdriver.Chrome("chromedriver", chrome_options=chrome_options)

        #driver = webdriver.Chrome(chrome_driver, chrome_options=chrome_options)

        if row['Hospital'] in IGNORE_HOSPITALS:
            # skip the entry
            #print(f'IGNORE {index}, {row["Hospital"]}')
            continue

        
        #driver.execute_script("window.open('');")
        #time.sleep(3)
        driver.switch_to.window(driver.window_handles[-1])
        time.sleep(2)
        driver.get(target_url)
        #driver.execute_script(f'''window.open("{target_url}","_blank");''')
        #time.sleep(3)
        #driver.switch_to.window(driver.window_handles[-1])

        # sleep 3 minutes to finish loading and scraping of page (hopefully reasonable) and downloading results
        #time.sleep(180)
        # replacing with something more dynamic and quicker process
        #if path.exists(download_file) and path.isfile(download_file):
        if wait_for_extension_data_output(download_file):
            driver.switch_to.window(driver.window_handles[0]) # switch back
            time.sleep(3)
            # move and rename file
            shutil.move(download_file, destination_file)
            print(f'\tloaded and parsed {driver.title}')
            print(f"\tfound {destination_file}")
            hit_exception = False
            time.sleep(2)
        else:
            print(f'\tGOT EXCEPTION={target_url}')
            driver.switch_to.window(driver.window_handles[0])
            time.sleep(2)
            delete_cache(driver)
            time.sleep(2)
            IGNORE_HOSPITALS.append(row['Hospital'])
            #raise ValueError("\t\t%s isn't a file!" % download_file)
            hit_exception = True
            #break

    #if not hit_exception:
    #    done = True
    #    hit_exception = False
    #    break
    break
# sleep 1 minute
#time.sleep(60)
#restart driver
driver.close()
driver.quit()


In [ ]:
# https://health.usnews.com/best-hospitals/area/ga/northside-hospital-6380171/doctors

In [ ]:
delete_cache()

In [ ]:
import webbrowser

In [ ]:
# INSPIRATION: https://pythonexamples.org/python-open-url-in-chrome-browser/
url = 'https://health.usnews.com/best-hospitals/area/az/mayo-clinic-6860019/doctors'
webbrowser.register('chrome', None, webbrowser.BackgroundBrowser("Google Chrome"))
webbrowser.get('chrome').open(url)
# webbrowser.get('chrome').open_new(url)
# https://health.usnews.com/best-hospitals/area/tn/vanderbilt-university-medical-center-6521060/doctors

In [ ]:
# vi ./System/Volumes/Data/Users/rajivdulepet/opt/anaconda3/pkgs/python-3.9.7-h88f2d9e_1/lib/python3.9/webbrowser.py
# /Users/rajivdulepet/Library/Application Support/Google/Chrome/Default


In [ ]:
# Try to open the file same file again
try:
    if path.exists(download_file) and path.isfile(download_file):
        with open(download_file, "r") as file:
            # Print the success message
            print("File has opened for reading.")
    # Raise error if the file is opened before
except IOError:
    print("File has opened already.")

In [ ]:
download_file